```
This software is part of GPU Ocean. 

Copyright (C) 2019  SINTEF Digital

This notebook sets up a simple example for testing/demonstrating the 
Observation class for writing and reading drifter observations to file.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Using the Observation class

This notebook sets up a simple example for testing/demonstrating the 
Observation class for writing and reading drifter observations to file.

## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import os
import sys
from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common, IPythonMagic

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import Observation
from SWESimulators import DataAssimilationUtils as dautils


In [ ]:
%cuda_context_handler gpu_ctx

# Create a small ensemble

Just copying in some old messy code for that.

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
initOption = 3
if initOption == 1:
    # Original initial conditions
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 2:
    # Initial conditions used for the SIR filter
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5
elif initOption == 3:
    # Initial conditions random - see further down!
    pass
    

if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print ("q0: ", q0)
print ("[f, g, H]", [f, g, waterDepth])
print ("f/gH: ", f/(g*waterDepth))
print ("gH/f: ", g*waterDepth/f)

reload(CDKLM16)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
sim = CDKLM16.CDKLM16(gpu_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)
if initOption == 3:
    sim.perturbState(q0_scale=100)

ensemble_size = 4
ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(gpu_ctx, ensemble_size, sim,
                                                 num_drifters=9,
                                                 observation_type=dautils.ObservationType.UnderlyingFlow,
                                                 #observation_type=dautils.ObservationType.DirectUnderlyingFlow,
                                                 observation_variance = 0.01,#**2,
                                                 initialization_variance_factor_ocean_field=20)


ensemble.step(ensemble.dt*100)

In [ ]:
ensemble.plotEnsemble()

# Register observations 
Register observations from the true state both through the old "live" methods, and through the Observation class

In [ ]:
reload(Observation)

## Compare Observation class with observations from ensemble:
new_obs = Observation.Observation()

# Make initial observations
new_obs.add_observation_from_sim(ensemble.particles[ensemble.obs_index])
ensemble.observeTrueState()

tmp_obs_from_ensemble = ensemble.observeTrueState()

iterations = 2
ensemble_obs = [None]*iterations
for i in range(iterations):
    ensemble.step(ensemble.dt*100)
    ensemble_obs[i] = ensemble.observeTrueState()
    new_obs.add_observation_from_sim(ensemble.particles[ensemble.obs_index])
print("ok")

In [ ]:
# The internal DataFrame now looks like:
new_obs.obs_df

# Compare old and new observations
Comparing observations from the ensemble with the observations from the Observation class.
The results should be exactly zero in the first two columns, and approximately zero in the last two.

In [ ]:
obs_times = new_obs.get_observation_times()
print(obs_times)
for i in range(iterations):
    #print(new_obs.obs_df.time.values[i])
    print(ensemble_obs[i] - new_obs.get_observation(obs_times[i], waterDepth))
print("ok")

# Write the observations to file

In [ ]:
filename='ObsClassTest.pickle'
new_obs.to_pickle(filename)

# Read the observations from file to a new Observation object

In [ ]:
read_obs = Observation.Observation()
read_obs.read_pickle(filename)

# Comparing the Observations read from file with the observations from the ensemble

In [ ]:
obs_times = read_obs.get_observation_times()
print(obs_times)
for i in range(iterations):
    print(ensemble_obs[i] - read_obs.get_observation(obs_times[i], waterDepth))
print("ok")